# Cycling power data

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor

In [ ]:
df = pd.read_csv('../.assets/data/cycling/cycling.csv')

In [ ]:
df.head()

## Description of the dataset

Timeseries of location and telemetry data containing:

* ```time```: time (UTC)
* ```lon```: position geographic longitude
* ```lat```: position geographic lattitude
* ```height```: position geographic height over sealevel
* ```power```: rider power output / W
* ```hr```: rider heartrate / bpm
* ```d```: cumulated distance travelled
* ```dist```: same as ```d```
* ```t```: ellapsed time / s
* ```x```: lateral position / m in reference frame
* ```y```: longitudal position / m in reference frame
* ```dxy```: distance between datapoints from cartesic coordinates
* ```newlap```: marker for beginning of new lap
* ```lap```: lap number

## A shape, well known by motorsport enthusiasts

In [ ]:
plt.scatter(df.lon, df.lat)

## The relationship between power and heartrate

If you press harder, your heartrate goes up. Sometimes it is as simple as that. So we should be able to see that in the data, right?

In [ ]:
plt.figure(dpi=200, figsize=(8,4))

plt.plot(df.hr)
plt.xlabel('elapsed time / s')
plt.ylabel('heartrate / bpm')

plt.twinx().plot(df.power, c='C1')
plt.ylabel('power / W')


## The power of noise

Well, in most cases real measurements have so surprises for us. Sometimes they look like noise, sometimes they actually are… Let's investigate this case and take a closer look.

In [ ]:
pf = df[df.lap == 1] # let's call this a plotframe

plt.figure(dpi=200, figsize=(8,4))

plt.plot(pf.hr)
plt.xlabel('elapsed time / s')
plt.ylabel('heartrate / bpm')

plt.twinx().plot(pf.power, c='C1')
plt.ylabel('power / W')

## Applying some smoothing

The almighty trick is smoothing the data. That can often be done by the means of rolling averages. Applying these only means, that in a timeseries at any give point in time you look at the average of the previous n values. Be careful though to not include data from the future when applying this  to your ML input data.

In [ ]:
pf = df[df.lap == 1] # let's call this a plotframe

plt.figure(dpi=200, figsize=(8,4))

plt.plot(pf.hr.rolling(60).mean(), label='hr')
plt.xlabel('elapsed time / s')
plt.ylabel('heartrate / bpm')

plt.legend()

plt.twinx().plot(pf.power.rolling(120, min_periods=1).mean(), c='C1', label='power')
plt.ylabel('power / W')

plt.legend()

## Dealing with offsets

In our sample the heartrate data seems to be slightly off from the power data. This can be explained by the cardial system responding to the change in power. Therefore, we can assume a _causal_ connection between the two variables. A change in power leads to a change in the heartrate.

We will use an auto-correlation approach to estimate the size of the offset.

In [ ]:
def calc_auto_correlation_shift(a,b):
    max_corr = 0
    pos = 0
    
    for i in range(len(b)):
        corr = np.correlate(a, np.roll(b,i))
        if np.abs(corr) > np.abs(max_corr):
            max_corr = corr
            pos = i
    
    return corr/len(b),pos

## Apply the auto correlation to our hr and power data

In [ ]:
a = pf.hr.rolling(60, min_periods=1).mean().values
b = pf.power.rolling(60, min_periods=1).mean().values

calc_auto_correlation_shift(a,b)

## Shift the data before plotting

In [ ]:
pf = df[df.lap == 1] # let's call this a plotframe

plt.figure(dpi=200, figsize=(8,4))

plt.plot(pf.hr.rolling(60, center=True).mean(), label='hr')
plt.xlabel('elapsed time / s')
plt.ylabel('heartrate / bpm')

plt.legend()

plt.twinx().plot(pf.power.rolling(60, center=True, min_periods=1).mean().shift(33), c='C1', label='power')
plt.ylabel('power / W')

plt.legend()

## Scatter plots for visualisations of correlation

In [ ]:
plt.scatter(
    pf.hr.rolling(60, center=True).mean(),
    pf.power.rolling(60, center=True).mean()
)
plt.scatter(
    pf.hr.rolling(60, center=True).mean(),
    pf.power.rolling(60, center=True).mean().shift(33)
)

In [ ]:
pf = df[df.lap==10]

r_hr = 60
r_power = 60

a = pf.hr.rolling(r_hr, min_periods=1).mean().values
b = pf.power.rolling(r_power, min_periods=1).mean().values

cor,shift = calc_auto_correlation_shift(a,b)

print(shift)

plt.figure(dpi=200, figsize=(8,4))

plt.plot(pf.hr.rolling(r_hr, center=True).mean(), label='hr')
plt.xlabel('elapsed time / s')
plt.ylabel('heartrate / bpm')

plt.legend()

plt.twinx().plot(pf.power.rolling(r_power, center=True, min_periods=1).mean().shift(shift), c='C1', label='power')
plt.ylabel('power / W')

plt.legend()

# Machine Learning

In [ ]:
model = RandomForestRegressor()

## A helper function to create datasets

In [ ]:
def prepare_df(df, lap=1, shift=0):
    train_df = df[df.lap==lap][['hr','power']].fillna(method='ffill').copy()
    print(lap, len(train_df))
    
    train_df['p030'] = df.power.rolling(30).mean()
    train_df['p060'] = df.power.rolling(30).mean()
    train_df['p300'] = df.power.rolling(30).mean()
    train_df['p600'] = df.power.rolling(30).mean()

    train_df['s030'] = df.power.rolling(30).std()
    train_df['s060'] = df.power.rolling(30).std()
    train_df['s300'] = df.power.rolling(30).std()
    train_df['s600'] = df.power.rolling(30).std()
    
    train_df['target'] = train_df.hr.shift(shift).astype(float)
    train_df = train_df.dropna()
    
    return train_df

## Step1: Just power and heartrate

In [ ]:
train_df = prepare_df(df, 1, shift=0)

X_train = train_df.power.values
y_train = train_df.target

In [ ]:
model.fit(X_train.reshape(-1, 1),y_train)

In [ ]:
y_pred = model.predict(X_train.reshape(-1, 1))

### Plot the results

In [ ]:
plt.figure(figsize=(8,4), dpi=100)

plt.plot(pd.Series(y_pred).rolling(60).mean())
plt.plot(pd.Series(y_train).rolling(60).mean().values)

## Step 2: including more variables

In [ ]:
X_train = train_df.drop(['hr','target'], axis=1).values
y_train = train_df.target

In [ ]:
model.fit(X_train,y_train)

### Apply the model to the _training_ data

In [ ]:
y_pred = model.predict(X_train)

In [ ]:
plt.figure(figsize=(8,4), dpi=100)

plt.plot(pd.Series(y_pred).rolling(60).mean())
plt.plot(pd.Series(y_train).rolling(60).mean().values)

### Get some independent test data and apply the model

In [ ]:
test_df = prepare_df(df, 3, shift=0)
X_test = test_df.drop(['hr','target'], axis=1).values
y_test = test_df.target

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
plt.figure(figsize=(8,4), dpi=100)

window=60

plt.plot(pd.Series(y_pred).rolling(window).mean())
plt.plot(pd.Series(y_test).rolling(window).mean().values)

plt.xlabel('time / s')

## A validation approach

In [ ]:
fig, axes = plt.subplots(1,2,dpi=100,figsize=(8,4))

axes[0].hist(
    pd.Series(y_pred).rolling(window).mean().values-
    pd.Series(y_test).rolling(window).mean().values,
    bins=np.linspace(-50,50,61)
);
axes[0].set_title(f'Rolling {window}')

axes[1].hist(
    pd.Series(y_pred).values-
    pd.Series(y_test).values,
    bins=np.linspace(-50,50,61)
);
axes[1].set_title('Raw data')

## Try yourself

### Improve the prediction

In [ ]:
#Your code goes here




### Predict the current heartrate

In [ ]:
#Your code goes here




### Predict the future heartrate

* Think about using additional data

In [ ]:
#Your code goes here


